# DM (device manufacturer) connects to music (device) and assign them ownership - Music

## 3. Initialize Music

In [6]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"
API_KEY = "bob_api_123456789"

agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


######################
# Remember to update #
######################

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'f01983a3-5d75-4a07-bcec-32c0af52b789', 'recipientKeys': ['8yDBAPXWfTEKdKCdRHBr7rKVpJ5pZkQ2bLGoJEXFbxCX'], 'label': 'Alice', 'serviceEndpoint': 'http://192.168.65.3:8020'}


# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0xffff8d6bf4c0>


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections
Connection Handler Called
Connection 5d918450-a06e-4cf6-a711-56d85333a1dd in State invitation
Connection Handler Called
Connection 5d918450-a06e-4cf6-a711-56d85333a1dd in State request
Connection {'their_label': 'Alice', 'rfc23_state': 'request-sent', 'their_role': 'inviter', 'updated_at': '2023-01-03 06:12:18.950337Z', 'invitation_mode': 'once', 'accept': 'manual', 'invitation_key': '8yDBAPXWfTEKdKCdRHBr7rKVpJ5pZkQ2bLGoJEXFbxCX', 'request_id': '7efa89d7-6529-4a47-9c26-db47d85014bf', 'my_did': 'YTZXU7cCnfKDw3cCqUhDbA', 'routing_state': 'none', 'connection_id': '5d918450-a06e-4cf6-a711-56d85333a1dd', 'created_at': '2023-01-03 06:12:18.896817Z', 'state': 'request'}


In [7]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'their_label': 'Alice', 'rfc23_state': 'request-sent', 'their_role': 'inviter', 'updated_at': '2023-01-03 06:12:18.950337Z', 'invitation_mode': 'once', 'accept': 'manual', 'invitation_key': '8yDBAPXWfTEKdKCdRHBr7rKVpJ5pZkQ2bLGoJEXFbxCX', 'request_id': '7efa89d7-6529-4a47-9c26-db47d85014bf', 'my_did': 'YTZXU7cCnfKDw3cCqUhDbA', 'routing_state': 'none', 'connection_id': '5d918450-a06e-4cf6-a711-56d85333a1dd', 'created_at': '2023-01-03 06:12:18.896817Z', 'state': 'request'}
State: request
Connection Handler Called
Connection 5d918450-a06e-4cf6-a711-56d85333a1dd in State response
Connection Handler Called
Connection 5d918450-a06e-4cf6-a711-56d85333a1dd in State active


# Give MUSIC the VC of ownership and origin - DM

## 3. Credential Exchange!

In [8]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 75c7d31c-9b2c-4f80-9929-13a7871f4946, role: holder, state: credential_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


In [9]:
# Request credential from issuer
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

ClientResponseError: 400, message='Credential exchange 75c7d31c-9b2c-4f80-9929-13a7871f4946 in credential_received state (must be offer_received).', url=URL('http://bob-agent:8051/issue-credential/records/75c7d31c-9b2c-4f80-9929-13a7871f4946/send-request')

Error during POST /issue-credential/records/75c7d31c-9b2c-4f80-9929-13a7871f4946/send-request: 400, message='Credential exchange 75c7d31c-9b2c-4f80-9929-13a7871f4946 in credential_received state (must be offer_received).', url=URL('http://bob-agent:8051/issue-credential/records/75c7d31c-9b2c-4f80-9929-13a7871f4946/send-request')


In [10]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 75c7d31c-9b2c-4f80-9929-13a7871f4946, role: holder, state: credential_received


In [11]:
# Store credential
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 75c7d31c-9b2c-4f80-9929-13a7871f4946, role: holder, state: credential_acked


## Remove connection !!!

In [9]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)
response = await agent_controller.connections.get_connections()
print(response)

{}
{'results': [{'their_label': 'Alice', 'accept': 'manual', 'state': 'request', 'connection_id': '521d9010-2f24-4a6b-9f48-c369db186f0b', 'invitation_key': 'GC6DHU7HHN4YVKoAHqrNg3VkpTyWUTpY7zKmpZSYvW1z', 'routing_state': 'none', 'created_at': '2023-01-02 13:20:47.668037Z', 'updated_at': '2023-01-02 13:20:47.728024Z', 'their_role': 'inviter', 'request_id': '18a7c875-3cf3-450e-92b6-850ddad8d0b8', 'invitation_mode': 'once', 'rfc23_state': 'request-sent', 'my_did': '2sTtp6vonk2F6gGjQELqFD'}, {'their_label': 'Alice', 'accept': 'manual', 'state': 'response', 'connection_id': '663ddab5-2135-478a-b7f1-3d7dfebe22ad', 'invitation_key': 'HHTzqiXeQey1NmZCZRkGMbT8KejyxGna2Su3gX3kHYDw', 'routing_state': 'none', 'created_at': '2023-01-02 13:21:20.734655Z', 'updated_at': '2023-01-02 13:21:38.481793Z', 'their_role': 'inviter', 'their_did': 'VWHUDyJon3XHkwdGiF3uQ9', 'request_id': '2f6713ee-3a34-44c8-b262-d0f9e955037a', 'invitation_mode': 'once', 'rfc23_state': 'response-received', 'my_did': '7vbdgKJzN44

In [12]:
response = await agent_controller.terminate()
print(response)

None
